In [2]:
import lancedb
from langchain.vectorstores import LanceDB
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import LanceDB
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())



In [2]:
!pip install gdown

  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/5e/5d/97afbafd9d584ff1b45fcb354a479a3609bd97f912f8f1f6c563cb1fae21/filelock-3.12.4-py3-none-any.whl.metadata


In [ ]:
import gdown
gdown.download_folder(
    "https://drive.google.com/drive/folders/1SkI0ttpMNTVHp6ear6cTLDooRmtqvmVo?usp=sharing",
      quiet=True,
      output="./") 

In [4]:
db = lancedb.connect('../.lancedb')
table = db.open_table('hf_docs')
# TODO: Create an OpenAIEmbeddings object and a LanceDB vectorstore instantiated with the table and OpenAIEmbeddings.
embedding_fn = OpenAIEmbeddings(chunk_size=200)
vectorstore = LanceDB(table, embedding_fn)

In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
#TODO: Create a RetrievalQA object with the ChatOpenAI model, the vectorstore's retriever, and a chain_type of "stuff".
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model='gpt-3.5-turbo-16k')
    , chain_type="stuff", retriever=vectorstore.as_retriever(search_kwargs=dict(k=5)),
      verbose=True, return_source_documents = True)

In [12]:
query = "Can you describe PEFT from the transformers library?"
answer = qa({"query": query})



> Entering new RetrievalQA chain...

> Finished chain.


In [8]:
answer

{'query': 'Can you describe computer vision in the transformers library?',
 'result': 'In the Transformers library, computer vision tasks are supported through various models and pipelines. The library provides pre-trained models for image classification, object detection, and image segmentation tasks. These models are based on Transformer architectures, which have been proven to be effective in natural language processing tasks and have been adapted for computer vision tasks as well.\n\nFor image classification, the library offers models like Vision Transformer (ViT) and Convolutional Vision Transformer (CvT). These models can take an image as input and classify it into different categories or labels.\n\nFor object detection, the library provides models like DETR (DEtection TRansformer) and YOLO (You Only Look Once). These models can detect and localize objects within an image, providing bounding box coordinates and class labels for each detected object.\n\nFor image segmentation, the

In [14]:
print(answer['result'])

PEFT, or Parameter-Efficient Fine-Tuning, is a method implemented in the Transformers library. It is designed to efficiently adapt pre-trained language models (PLMs) to various downstream tasks without fine-tuning all of the model's parameters.

PEFT methods only fine-tune a small number of additional parameters, significantly reducing computational and storage costs compared to traditional fine-tuning. This is particularly important for large-scale PLMs, which have a large number of parameters that make full fine-tuning prohibitively expensive.

PEFT methods, such as LoRA (Low-Rank Adaptation), achieve performance comparable to full fine-tuning while using significantly fewer parameters. This makes it possible to train and deploy models that would otherwise be too large for consumer devices.

The Transformers library provides a utility library for PEFT called "PEFT as a utility library". It allows users to leverage PEFT's low-level API to inject trainable adapters into any torch modul

## Experiment: Retrieval-Augmented Generation vs Vanilla Generation

TODO: Write a python snippet that creates an llm using the ChatOpenAI object with a temperature of 0, and prompt it with `query`. Then, print the output of the llm. `query` is a string that contains the query you want to prompt the llm with and it should ask the language model a question related to the Huggingface library.

In [15]:
# TODO: Write your code here.
llm=ChatOpenAI(temperature=0, model='gpt-3.5-turbo-16k')
print(llm.predict(query))

PEFT (Pretraining with Extracted Feature-based Training) is a method introduced in the transformers library for pretraining language models. It aims to improve the efficiency and effectiveness of pretraining by leveraging extracted features from a pretrained model.

In traditional pretraining, models are trained to predict masked tokens in a sentence, which requires a large amount of computation and data. PEFT, on the other hand, reduces the computational cost by using a pretrained model to extract features from a large unlabeled dataset. These features are then used to train a smaller model, known as the "head" model, to predict the masked tokens.

The PEFT process involves the following steps:

1. Pretraining a large model: A large model, such as BERT, is pretrained on a large corpus using the standard masked language modeling objective. This model serves as the "teacher" model.

2. Feature extraction: The teacher model is used to extract features from a large unlabeled dataset. Thes

In [ ]:
query = "Can you give me a crash course on using the HF Trainer to train a GPT-style model?"
answer = qa({"query": query})
print(answer['result'])

In [18]:
query = "Can you give me a crash course on using the HF Trainer to train a GPT-style model? Provide code snippets for each step."
answer = qa({"query": query})
print(answer['result'])



> Entering new RetrievalQA chain...

> Finished chain.


## Observations and Insights
Write any observations here.

## Using Internet Search Engines to Augment Retrieval

Hypothesis: This will be more useful than the previous method for "how to" and "crash course" requests, because it will allow us to retrieve more relevant documents from the internet.

In [ ]:
!pip install -q google-api-python-client
!pip install -q chromadb

## Setup

1. To create an API key: Go to https://console.cloud.google.com/welcome/new and sign in/up using your personal Google account. 
   - Navigate to the APIs & Services→Credentials panel in Cloud Console. 
   - Select Create credentials, then select API key from the drop-down menu. 
   - The API key created dialog box displays your newly created key. 
   - You now have an API_KEY. Store in it in your `.env` file as `GOOGLE_API_KEY`.

2. Setup Custom Search Engine so you can search the entire web 
   - Create a custom search engine in this [link](http://www.google.com/cse/). 
   -  In Sites to search, select search the entire web. Select also enable SafeSearch and fill in your search engine name (it can be anything).
   - Click on customize.
   - Under Search engine ID you’ll find the search-engine-ID. Copy that and add it to your `.env` file as `GOOGLE_CSE_ID`.

3. Enable the Custom Search API 
   - Go to this [URL](https://console.cloud.google.com/apis/library/customsearch.googleapis.com) & click on Enable.
   - Alternatively, navigate to the APIs & Services→Dashboard panel in Cloud Console. 
   - Click Enable APIs and Services. 
   - Search for Custom Search API and click on it. 
   - Click Enable. 

**Note**: Adapted from [GoogleSearchAPIWrapper Docs](https://api.python.langchain.com/en/latest/utilities/langchain.utilities.google_search.GoogleSearchAPIWrapper.html)

In [5]:
#TODO: Re-load the environment variables again
_ = load_dotenv(find_dotenv())

In [21]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.vectorstores import Chroma
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

# Vectorstore
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="./chromadb_oai")

# LLM
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-16k', max_tokens=512)

search = GoogleSearchAPIWrapper()

In [22]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm, 
    search=search,
)

In [1]:
from langchain.chains import RetrievalQAWithSourcesChain
query = "How can I finetune a Llama2 7b model on Google Colab?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm, retriever=web_research_retriever)
result = qa_chain({"question": query})
result

NameError: name 'llm' is not defined

## Observations and Insights
Write any observations here.

How can we make it better? We want to be able to retrieve more relevant documents from the internet but also generate complete answers to the user's question without having the user to read the retrieved documents.

## References
* https://lancedb.github.io/lancedb/notebooks/code_qa_bot/
* https://github.com/langchain-ai/chat-langchain/tree/master